## Завдання 1.1

1. В чому різниця запитів?
2. Чим будуть відрізнятись отримані результати?

_*1-й Запит:*_\
SELECT * \
FROM table1 LEFT JOIN table2 USING (name)\
WHERE table2.name = 'John'

_*2-й Запит:*_\
SELECT * \
FROM table1 JOIN table2 USING (name)\
WHERE table2.name = 'John'

#### Відповіді:

__1. У першому  запиті використовується LEFT JOIN, який повертаіє всі рядки з table1 та відповідні з table2, а  JOIN повертає лише ті дані, що співпадають в обох таблицях.__

__2. Ні, адже після об'єднання виконується фільтрація *WHERE table2.name = 'John'*,  тому всі рядки з нульовими значеннями, які могли бути після використання LEFT JOIN.__

## Завдання 1.2
*Написати SQL скрипт, що витягує всіх юзерів, total spend яких вище ніж середній ТС всіх юзерів в розрізі регіонів у [датасеті](https://docs.google.com/spreadsheets/d/1B9Biy7gYjstobq2CVD5o_rCIObEFTLypff5JsSQB1Ng/edit?gid=0#gid=0)*

#### Завнатаження необхідних бібліотек для виконнання завдання:


In [21]:
import pandas as pd
import sqlite3

#### Завантаження таблиці та попередній огляд даних:

In [29]:
df = pd.read_csv("task_1.csv")
df.head()

,id_order,id_user,status,date_created,amount,id_region
0,7391a43e-142d-49f0-91bd-e03c23e8de48,87630,success,2023-09-21 19:06:17 UTC,2,1
1,87be7485-0dbc-4fc3-8f3d-ba8f46f3b1c7,87630,success,2023-09-21 19:40:17 UTC,"4,5",1
2,64caed80-ddc7-4f77-b156-7b2fc772b6b8,82145,fail,2023-09-21 09:11:05 UTC,2,1
3,9dd77e41-7bd0-4ae3-8248-39b3cc789413,82145,fail,2023-09-21 15:21:15 UTC,2,1
4,c07c0bd1-eea4-457b-b89d-89b5945d879e,82145,fail,2023-09-21 16:31:17 UTC,2,1


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49817 entries, 0 to 49816
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_order      49817 non-null  object
 1   id_user       49817 non-null  int64 
 2   status        49817 non-null  object
 3   date_created  49817 non-null  object
 4   amount        49817 non-null  object
 5   id_region     49817 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.5+ MB


Перевірка на наявність дублікатів:

In [24]:
df.duplicated().sum()

np.int64(0)

#### Робота з SQL запитом 
Створення бази даних та завнтаження до неї таблиці:

In [25]:
conn = sqlite3.connect(":memory:") 
df.to_sql("task_1", conn, index=False, if_exists="replace")

49817

#### SQL запит:

In [26]:
query = """
WITH 
    user_totals AS (
        SELECT 
          id_user, 
          SUM(amount) OVER (PARTITION BY  id_user) AS total_spend, 
          id_region
        FROM task_1
        WHERE status	= "success"
        GROUP BY  id_user
),
    region_avg AS (
         SELECT 
           id_region,
           AVG(total_spend) OVER (PARTITION BY id_region) AS avg_total_spend      
         FROM user_totals
         GROUP BY id_region
)    
SELECT 
    ut.id_user,
    ut.total_spend,
    ra.avg_total_spend AS region_avarage,
    ut.id_region
       
FROM user_totals AS ut JOIN region_avg AS ra USING(id_region)
WHERE ut.total_spend > ra.avg_total_spend;
"""

##### CTE:
  * user_totals -  розрахунок ТС для кожного користувача, де змавлення зі сатусом "success",
  * region_avg - розрахунок середнього ТС для кожного з регіонів.

##### Основний SELECT: 
  * обєднуючи user_totals та region_avg, виводить юзерів, у яких  ТС більший за середній ТС у розрізі його регіону.

 Виведення результату запиту:
 

In [27]:
pd.read_sql_query(query, conn)

,id_user,total_spend,region_avarage,id_region
0,4817,200.0,80.0,1
1,5222,200.0,80.0,1
2,5370,200.0,80.0,1
3,6767,200.0,80.0,2
4,7065,200.0,80.0,2
...,...,...,...,...
961,87572,12.0,4.0,5
962,87579,80.0,4.0,5
963,87601,15.0,4.0,5
964,87609,15.0,4.0,5


Завершення роботи з базою даних:

In [28]:
conn.close()